In [1]:
import os
os.chdir('../..')

In [193]:
import pandas as pd
import numpy as np

from datetime import datetime, date
import matplotlib.pyplot as plt

import json

data = 'data/'

In [4]:
atms = pd.read_csv(data + 'atms.small.csv')
clients = pd.read_csv(data + 'clients.small.csv')
companies = pd.read_csv(data + 'companies.small.csv')
transactions = pd.read_csv(data + 'transactions.small.csv')

In [66]:
months = {}
days = {}
counts = 0
dates = []
for i in transactions.index:
    date = transactions.loc[i]['date']
    month = int(date.split('/')[1])
    day = int(date.split('/')[0])
    
    if month not in months.keys():
        months[month] = 0
    
    months[month] += 1
    if day not in days.keys():
        days[day] = 0
    
    days[day] += 1
    
    if day <= 12:
        counts += 1
    else:
        dates.append(date)

In [65]:
counts/len(transactions)

0.8444396365209866

In [54]:
months

{1: 618,
 2: 622,
 3: 663,
 4: 527,
 5: 518,
 6: 661,
 7: 564,
 8: 570,
 9: 627,
 10: 450,
 11: 505,
 12: 507,
 13: 759,
 14: 552,
 15: 575,
 16: 612,
 17: 843,
 18: 675,
 19: 579,
 20: 922,
 21: 612,
 22: 709,
 23: 752,
 24: 556,
 25: 503,
 26: 474,
 27: 646,
 28: 515,
 29: 558,
 30: 579,
 31: 235}

In [57]:
days

{1: 1246,
 2: 1301,
 3: 1188,
 4: 1265,
 5: 1283,
 6: 1368,
 7: 1241,
 8: 1317,
 9: 1307,
 10: 1270,
 11: 1284,
 12: 1542,
 13: 49,
 14: 52,
 15: 43,
 16: 49,
 17: 46,
 18: 40,
 19: 45,
 20: 45,
 21: 37,
 22: 44,
 23: 41,
 24: 233,
 25: 524,
 26: 573,
 27: 1055}

## Extract time patterns

In [117]:
threshold_minutes = 10
threshold_amount = 0.1

In [182]:
groups = {'source': {'time': [], 'flow': []},
          'target': {'time': [], 'flow': []}}

ids = {'source': transactions['source'].unique(), 'target': transactions['target'].unique()}

for key in ids.keys():
    print('Check for {}'.format(key))
    for id_ in ids[key]:
        df = transactions[transactions[key] == id_]
        df.index = range(len(df))

        times = np.zeros((len(df), len(df)), dtype=np.bool)
        amounts = np.zeros((len(df), len(df)), dtype=np.bool)


        for j in range(len(df)):
            for k in range(j, len(df)):
                # Time diff
                tj = datetime.strptime(df.loc[j]['time'], '%H:%M:%S').time()
                tk = datetime.strptime(df.loc[k]['time'], '%H:%M:%S').time()

                diff = np.abs(((datetime.combine(date.today(), tj) - datetime.combine(date.today(), tk)).total_seconds()))

                if diff < threshold_minutes*60:
                    times[j][k] = True
                    times[k][j] = True
                else:
                    times[j][k] = False
                    times[k][j] = False

                # Amount diff

                amj = df.loc[j]['amount']
                amk = df.loc[k]['amount']

                diff = np.abs(amj-amk)
                avg = np.mean([amj, amk])

                if diff < threshold_amount*avg:
                    amounts[j][k] = True
                    amounts[k][j] = True
                else:
                    amounts[j][k] = False
                    amounts[k][j] = False

        mat = amounts*times

        j=0
        while j <= len(df)-1:
            arr = [i for i, x in enumerate(mat[j, :]) if x]
            j = arr[-1]+1
            if len(arr) > 1:
                linked = [df.loc[k]['id'] for k in arr]
                if len(df[df['id'].isin(linked)]['date'].unique()) == 1:
                    groups[key]['flow'].append(linked)
                else:
                    groups[key]['time'].append(linked) 

Check for source
Check for target


In [186]:
for key in groups.keys():
    print('{}:'.format(key))
    for type_ in groups[key].keys():
        print('  {}: {}'.format(type_, len(groups[key][type_])))

source:
  time: 172
  flow: 287
target:
  time: 189
  flow: 284


In [191]:
189+284

473

In [192]:
transactions[transactions['id'].isin(groups['target']['flow'][0])]

,id,source,target,date,time,amount,currency
0,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
91,d3dd0c1f-8c3a-4f7c-b042-c30dee422866,b38c9ead-22da-4455-8f71-97a334b911a1,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
144,8d0c3f7f-4286-4122-a911-479d45b5570b,77262493-a17e-43e5-bab3-92daa34e2807,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
520,bd3920fe-f523-421e-ad43-6e7e2fd6cb3e,d845295f-70ef-4cca-b370-3b3adf817867,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
840,1d094f90-ea10-47cf-b520-1ea2b57e064e,29e199c2-2983-498e-bbbf-0695ec0466da,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,LKR
898,770d4f96-959a-45d5-a1bd-2a02a1e3db82,ae844c99-ccdc-4de7-a7a9-8fd09f4edbeb,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
1573,0587299a-c7bd-4a63-96ed-61ab71d9545d,963b1817-e2fb-45b7-a40f-67fc84c9b040,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,None
1863,7f5c8d79-1e54-411f-8cd1-86f62f3e830b,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,15:57:52,1683.730271,IQD


In [ ]:
with open(data + 'groups.json', 'w')